In [1]:
import gc
import sys
import time
import json
import logging
import numpy as np
import pandas as pd
from tqdm import tqdm
from imp import reload
from scipy import sparse
from joblib import Parallel, delayed

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.legend as mlegend
from gensim.models import Word2Vec

path = "/home/grace/walking/accelerate_discoveries" # path to the repository accelerate-discoveries
sys.path.insert(0,path)
import literature, embedding, utils, evaluation

%matplotlib inline

In [2]:
path_to_VM = "/home/grace/walking/VertMat.npz"   #  path to the vertex matrix file
VM = sparse.load_npz(path_to_VM)
path_to_matnames = "/home/grace/walking/mats_names.txt"   # path to the text file that contains our extracted material names
mats = open(path_to_matnames,'r', encoding = "utf-8").read().splitlines()
path_to_propnames = "/home/grace/walking/props_names.txt"  # path to the text file that contains properties we considered 
props = open(path_to_propnames,'r').read().splitlines()

In [3]:
# finally, time to create a hypergraph
h = literature.hypergraph(VM, mats, props)

# Running Random Walk Sampling

In [4]:
# ignore this line for now
h.Rcsr = h.R.tocsr()

# Training a Word2Vec

Now you can start playing with the output of the random walk sampling (e.g. train a word2vec on them,etc.). I will also add more stuff to this section later and share with you. 

### Preprocessing the Data

In [5]:
# load the random walk sequences
path_to_rws_L80x5M = "/home/grace/walking/nodes_L80_5M.txt"    # this should be the path to the text file that includes the random walk sequences 
rws_L80x5M = open(path_to_rws_L80x5M,'r', encoding = "utf-8").read().splitlines()

In [6]:
# the first step, would be to remove the author nodes from the node sequences
pruned_rws_L80x5M = utils.remove_authors_from_RW(rws_L80x5M)

# after removing authors, some sequences might have only one remaining node
# we remove those so that the resulting random walks would have at least two nodes (at least one step)
pruned_rws_L80x5M = [x for x in pruned_rws_L80x5M if len(x.split(' '))>1]

In [7]:
print("Number of all random walks: {}".format(len(rws_L80x5M)))
print("Number of random walks after removing authors: {}".format(len(pruned_rws_L80x5M)))
print("\nExample random walk: {}".format(pruned_rws_L80x5M[100]))

Number of all random walks: 23278
Number of random walks after removing authors: 23071

Example random walk: functional__groups/imine functional__groups/aromatic methylphenidate Beacon Beacon functional__role/dye disodium pro rifampicin clarithromycin isoniazid estrogen thymoquinone MDA amino__acid heme Hydrogen diphenhydramine__HCl acetaminophen HeH 2PA pyrene spermine xylose mannitol atropine Calcium cGMP biflavone C18:3 NaCl fructose imidacloprid γ-aminobutyric__acid citrate


In [8]:
# save it somewhere, because we will use the path to create the embedding model
path_to_pruned_rws_L80x5M = "/home/grace/walking/pruned_rws_L80x5M.txt" #file path to save the pruned random walks (author-less sequences)
open(path_to_pruned_rws_L80x5M,'w', encoding = "utf-8").write('\n'.join(pruned_rws_L80x5M)+'\n') # returns total number of lines
#open: 'w' open for writing

6832132

### Creating the Embedding Model

In [9]:
# instantiate a deepwalk-word2vec object: 
# we need to input the path to pruned random walks, the model will itself load the file later on
M = embedding.dww2v(path_to_pruned_rws_L80x5M,depth=0)

In [10]:
# if we don't provide any optional inputs, all parameters take their default values
M.pars

{'depth': 0,
 'phrase_min_count': 10,
 'phrase_threshold': 15,
 'size': 200,
 'window': 8,
 'min_count': 5,
 'sg': True,
 'hs': True,
 'workers': 20,
 'negative': 15,
 'start_alpha': 0.001,
 'end_alpha': 0.0001,
 'subsample': 0.0001,
 'batch': 5000,
 'epochs': 5}

In [11]:
# these few lines are just to specify how pythpn will display/store the logs
M.logger.handlers = []
logging.root.handlers = [logging.StreamHandler()]
logging.root.setLevel(logging.INFO)

Now let's create the embedding model. At this step, the model will use the following parameters:
* `depth`: a parameter specifying the number of tokens that can be included in a phrase. Note that in regular texts, it makes a lot of sense to work with phrases of various lengths, e.g., "bank_of_america", which is a phrase of length 3, but in deepwalk it is not as clear if phrasing will be useful. However, in our initial experiments, we found out that phrasing with `depth=2` improves the result. Setting `depth=0` (by running `embedding.dww2v(path_to_pruned_rws, depth=0)` at the time of instantiation or running `M.pars['depth']=0` after instantiation) will skip the phrasing step.
* `phrase_min_count`: A parameter that tells the function to ignore all words and bigrams with total collected count lower than this value
* `phrase_threshold`: Represents a score threshold for forming the phrases (higher means fewer phrases). This parameter is tightly connected to the built-in functions that gensim use to perform phrasing.

In [12]:
# this will take a few minutes, and display many lines of information
M.build_model()
path_to_M_L80x5M = "/home/grace/walking/M_L80x5M.model"
M.save_model(path_to_M_L80x5M)

Parsing lines (sentences) in: /home/grace/walking/pruned_rws_L80x5M.txt: 
Parameters for parsing phrases are as follows:
	depth: 0
	phrase_min_count: 10
	phrase_threshold: 15
Word2Vec lifecycle event {'params': 'Word2Vec(vocab=0, vector_size=200, alpha=0.001)', 'datetime': '2022-06-21T14:22:24.184899', 'gensim': '4.1.2', 'python': '3.9.12 (main, Apr  5 2022, 06:56:58) \n[GCC 7.5.0]', 'platform': 'Linux-5.13.0-48-generic-x86_64-with-glibc2.34', 'event': 'created'}
collecting all words and their counts
PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
PROGRESS: at sentence #10000, processed 251607 words, keeping 33617 word types
PROGRESS: at sentence #20000, processed 503066 words, keeping 48907 word types
collected 52596 word types from a corpus of 580812 raw words and 23071 sentences
Creating a fresh vocabulary
Word2Vec lifecycle event {'msg': 'effective_min_count=5 retains 10819 unique words (20.570005323598753%% of original 52596, drops 41777)', 'datetime': '2022-06-2

In [13]:
vocab_arr = np.array(list(M.model.wv.key_to_index))
print("Number of words in the model's vocabulary: {}".format(len(vocab_arr)))
print("\nExample words:\n{}".format(vocab_arr[np.random.RandomState(10).randint(0,len(vocab_arr),20)].tolist()))

Number of words in the model's vocabulary: 10819

Example words:
['As(III)', 'BTO', 'benzoic__acid', 'coumestrol', 'perchlorates', 'phosphatidylmonomethylethanolamine', 'hydroxysteroid', 'Cr(III)', 'GEP', 'ethionine', '2-methoxyethanol', 'cystamine', 'polyhedrin', 'HPA', 'PpIX', 'Vp', 'AMP', 'chlorhexidine__gluconate', 'crystalline__silica', 'acetoacetate']


At this time, the model is not trained yet. Let's train it for a few epochs. At this point, the model will use all the other parameters
* `size`: size of the embedding vector
* `window`: size of neighborhood in selecting the nearby words
* `min_count`: the model will ignore words with counts smaller than this threshold
* `sg`: to use **skipgram** word2vec or the other variant (i.e., CBOW) 
* `hs`: to use heirarchical softmax or not (this is a trick for training the model less expensively; you might not need to know details at this time, but if you are curious see the paper)
* `workers`: number of worker threads to use for training
* `negative`: number of negative samples to be used in negative sampling (this is another popular trick in training word2vec;you might not need to know details at this time, but if you are curious see the paper).
* `start_alpha`: initial learning rate
* `end_alpha`: the ending learning rate; the learning rate starts from `start_alpha` in the beginning of the training and linearly drops to `end_alpha` as the training progresses
* `subsample`: the threshold for configuring which higher-frequency words are randomly downsampled
* `batch`: number of words in the training batches of examples 
* `epochs`: number of training epochs

In [14]:
# run the training for five epochs (default value for parameter "epochs")
# (this also takes some time and prints a lot of information)
model = Word2Vec.load(path_to_M_L80x5M)
M.train()

loading Word2Vec object from /home/grace/walking/M_L80x5M.model
loading wv recursively from /home/grace/walking/M_L80x5M.model.wv.* with mmap=None
setting ignored attribute cum_table to None
Word2Vec lifecycle event {'fname': '/home/grace/walking/M_L80x5M.model', 'datetime': '2022-06-21T14:22:25.076392', 'gensim': '4.1.2', 'python': '3.9.12 (main, Apr  5 2022, 06:56:58) \n[GCC 7.5.0]', 'platform': 'Linux-5.13.0-48-generic-x86_64-with-glibc2.34', 'event': 'loaded'}
Training the model using the following parameters:
	phrase_min_count: 10
	size: 200
	window: 8
	min_count: 5
	sg: True
	hs: True
	workers: 20
	negative: 15
	start_alpha: 0.001
	end_alpha: 0.0001
	subsample: 0.0001
	batch: 5000
	epochs: 5
The model will be saved in None
Word2Vec lifecycle event {'msg': 'training model with 20 workers on 10819 vocabulary and 200 features, using sg=1 hs=1 sample=0.0001 negative=15 window=8 shrink_windows=True', 'datetime': '2022-06-21T14:22:25.083190', 'gensim': '4.1.2', 'python': '3.9.12 (main,

worker thread finished; awaiting finish of 9 more threads
worker thread finished; awaiting finish of 8 more threads
worker thread finished; awaiting finish of 7 more threads
worker thread finished; awaiting finish of 6 more threads
worker thread finished; awaiting finish of 5 more threads
worker thread finished; awaiting finish of 4 more threads
worker thread finished; awaiting finish of 3 more threads
worker thread finished; awaiting finish of 2 more threads
worker thread finished; awaiting finish of 1 more threads
worker thread finished; awaiting finish of 0 more threads
EPOCH - 5 : training on 580812 raw words (307257 effective words) took 3.3s, 94261 effective words/s
5 Epoch(s) done. Loss: 1192499.0, LR: 0.00010074899224134205
Word2Vec lifecycle event {'msg': 'training on 2904060 raw words (1536157 effective words) took 17.0s, 90507 effective words/s', 'datetime': '2022-06-21T14:22:42.056531', 'gensim': '4.1.2', 'python': '3.9.12 (main, Apr  5 2022, 06:56:58) \n[GCC 7.5.0]', 'plat

# Evaluating the Trained Model

In [15]:
props = {'state': ['crystalline', 'semicrystalline', 'amorphous', 'liquid'],
         'composition': ['organic', 'hybrid', 'inorganic', 'metallic'],
         'conjugation': ['conjugated', 'non-conjugated'],
         'porosity': ['microporous', 'mesoporous', 'non-porous'],
         'form': ['particles', 'nanoparticles', 'solid', 'liquid'],
         'hydrophilicity': ['hydrophobic', 'hydrophilic'],
         'dispersibility': ['dispersible',
                            'poorly__dispersible',
                            'non-dispersible',
                            'soluble'],
         'conductivity': ['conducting', 'semi-conducting', 'insulating'],
         'functional__role': ['light__absorber',
                              'cocatalyst',
                              'electron__scavenger',
                              'hole__scavenger',
                              'dye',
                              'solvent',
                              'surfactant',
                              'her__catalyst',
                              'oer__catalyst',
                              'overall__ws__catalyst',
                              'ph__modifier',
                              'dielectric__modifier'],
         'water__solubility': ['soluble', 'insoluble', 'sparingly__soluble'],
         'functional__groups': ['aromatic',
                                'aliphatic',
                                'carboxylic__acid',
                                'ester',
                                'imine',
                                'sulfone',
                                'triazine']
        }

## Evaluating Effect of Parameters

In [16]:
size_arange = np.arange(50,200+50,50)
window_arange = np.arange(2,20+2,2)
workers_arange = np.arange(10,40,5)
batch_arange = np.arange(1000,10000,1000)
epochs_arange = np.arange(2,10,1)

In [17]:
temp_path = '/home/grace/walking/temporary/N.model'
def preturb(size = 200, window = 8, workers = 20, batch = 5000, epochs = 5):
    M.pars['size'] = size
    M.pars['window'] = window
    M.pars['workers'] = workers
    M.pars['batch'] = batch
    M.pars['epochs'] = epochs
    M.train()
    M.save_model(temp_path)

In [18]:
mat_prop_tuples = [(x,y) for x in ["NaCl","H","glucose","acetone"]\
                  for y in ["state","composition","conjugation","porosity","form","hydrophilicity","dispersibiity","conductivity","functional__role","water__solubility","functional__groups"]]
def dict_to_list(dictionary):
    prop_infers_list = [dictionary[x] for x in dictionary]
    return prop_infers_list
def infer(N):
    infers = []
    for mat in ['NaCl','(1)H','glucose','acetone']:
        scores, prop_infers = evaluation.infer_props(N.model, mat, props)
        infers.append(dict_to_list(prop_infers))
    return(infers)

prop_tuples = []
for prop in props:
    for option in props[prop]:
        prop_tuples.append((prop,option))
# *!
def score(N, mat = 'NaCl'): # doesn't repeat trials for each i
    scores_dict, prop_infers = evaluation.infer_props(N.model, mat, props)
    scores = []
    for child in scores_dict.values(): # getting all the numbers in the dictionary
        for value in child.values():
            scores.append(value)
    return scores

In [36]:
# *!
evaluation.infer_props(size_models[50].model, 'NaCl', props) # example of where the scores came from

({'state': {'crystalline': -0.5145336985588074,
   'amorphous': -0.5253780484199524,
   'semicrystalline': -0.5304334163665771,
   'liquid': -0.5542548298835754},
  'composition': {'metallic': -0.4714057445526123,
   'inorganic': -0.5047813653945923,
   'hybrid': -0.5717554092407227,
   'organic': -0.5960980653762817},
  'conjugation': {'non-conjugated': -0.49916574358940125,
   'conjugated': -0.5855868458747864},
  'porosity': {'non-porous': -0.45811477303504944,
   'microporous': -0.49949270486831665,
   'mesoporous': -0.5109043717384338},
  'form': {'particles': -0.570732057094574,
   'liquid': -0.5732400417327881,
   'nanoparticles': -0.5927636027336121,
   'solid': -0.6251510381698608},
  'hydrophilicity': {'hydrophobic': -0.5300103425979614,
   'hydrophilic': -0.5512899160385132},
  'dispersibility': {'non-dispersible': -0.43539613485336304,
   'poorly__dispersible': -0.43550097942352295,
   'dispersible': -0.4802376329898834,
   'soluble': -0.5452079176902771},
  'conductivity':

In [33]:
df_size_infers = pd.DataFrame(index=pd.MultiIndex.from_tuples(mat_prop_tuples, names = ["mat","prop"]))
df_size_scores = pd.DataFrame(index=pd.MultiIndex.from_tuples(prop_tuples, names = ["prop","option"]))
size_models = {}
for i in size_arange:
    preturb(size = i)
    size_models[i] = embedding.dww2v(None)
    size_models[i].load_model(temp_path)
    
    infers = infer(size_models[i])
    df_size_infers["size = " + str(i)] = [prop for mat in infers for prop in mat] # turn lol into list
    
    df_size_scores["size = " + str(i)] = score(size_models[i])
df_size_scores["std"] = df_size_scores.std(axis = 1).tolist()

2022-06-22 22:05:16,989 : INFO : Training the model using the following parameters:
2022-06-22 22:05:16,989 : INFO : 	phrase_min_count: 10
2022-06-22 22:05:16,990 : INFO : 	size: 50
2022-06-22 22:05:16,990 : INFO : 	window: 8
2022-06-22 22:05:16,991 : INFO : 	min_count: 5
2022-06-22 22:05:16,991 : INFO : 	sg: True
2022-06-22 22:05:16,992 : INFO : 	hs: True
2022-06-22 22:05:16,992 : INFO : 	workers: 20
2022-06-22 22:05:16,992 : INFO : 	negative: 15
2022-06-22 22:05:16,993 : INFO : 	start_alpha: 0.001
2022-06-22 22:05:16,993 : INFO : 	end_alpha: 0.0001
2022-06-22 22:05:16,993 : INFO : 	subsample: 0.0001
2022-06-22 22:05:16,994 : INFO : 	batch: 5000
2022-06-22 22:05:16,994 : INFO : 	epochs: 5
2022-06-22 22:05:16,995 : INFO : The model will be saved in None
2022-06-22 22:05:16,995 : WARNING : Effective 'alpha' higher than previous training cycles
2022-06-22 22:05:16,996 : INFO : Word2Vec lifecycle event {'msg': 'training model with 20 workers on 10819 vocabulary and 200 features, using sg=

2022-06-22 22:05:27,919 : INFO : EPOCH 4 - PROGRESS: at 59.29% examples, 88807 words/s, in_qsize 40, out_qsize 0
2022-06-22 22:05:28,458 : INFO : worker thread finished; awaiting finish of 19 more threads
2022-06-22 22:05:28,522 : INFO : worker thread finished; awaiting finish of 18 more threads
2022-06-22 22:05:28,597 : INFO : worker thread finished; awaiting finish of 17 more threads
2022-06-22 22:05:28,613 : INFO : worker thread finished; awaiting finish of 16 more threads
2022-06-22 22:05:28,618 : INFO : worker thread finished; awaiting finish of 15 more threads
2022-06-22 22:05:28,635 : INFO : worker thread finished; awaiting finish of 14 more threads
2022-06-22 22:05:28,640 : INFO : worker thread finished; awaiting finish of 13 more threads
2022-06-22 22:05:28,641 : INFO : worker thread finished; awaiting finish of 12 more threads
2022-06-22 22:05:28,645 : INFO : worker thread finished; awaiting finish of 11 more threads
2022-06-22 22:05:28,657 : INFO : worker thread finished; aw

2022-06-22 22:05:34,797 : INFO : worker thread finished; awaiting finish of 11 more threads
2022-06-22 22:05:34,827 : INFO : worker thread finished; awaiting finish of 10 more threads
2022-06-22 22:05:34,860 : INFO : worker thread finished; awaiting finish of 9 more threads
2022-06-22 22:05:34,874 : INFO : worker thread finished; awaiting finish of 8 more threads
2022-06-22 22:05:34,876 : INFO : worker thread finished; awaiting finish of 7 more threads
2022-06-22 22:05:34,883 : INFO : worker thread finished; awaiting finish of 6 more threads
2022-06-22 22:05:34,887 : INFO : worker thread finished; awaiting finish of 5 more threads
2022-06-22 22:05:34,898 : INFO : worker thread finished; awaiting finish of 4 more threads
2022-06-22 22:05:34,905 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-06-22 22:05:34,908 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-06-22 22:05:34,912 : INFO : worker thread finished; awaiting finish of 1 more threa

2022-06-22 22:05:45,765 : INFO : EPOCH 5 - PROGRESS: at 57.58% examples, 87633 words/s, in_qsize 40, out_qsize 2
2022-06-22 22:05:46,364 : INFO : worker thread finished; awaiting finish of 19 more threads
2022-06-22 22:05:46,409 : INFO : worker thread finished; awaiting finish of 18 more threads
2022-06-22 22:05:46,461 : INFO : worker thread finished; awaiting finish of 17 more threads
2022-06-22 22:05:46,497 : INFO : worker thread finished; awaiting finish of 16 more threads
2022-06-22 22:05:46,510 : INFO : worker thread finished; awaiting finish of 15 more threads
2022-06-22 22:05:46,537 : INFO : worker thread finished; awaiting finish of 14 more threads
2022-06-22 22:05:46,548 : INFO : worker thread finished; awaiting finish of 13 more threads
2022-06-22 22:05:46,560 : INFO : worker thread finished; awaiting finish of 12 more threads
2022-06-22 22:05:46,605 : INFO : worker thread finished; awaiting finish of 11 more threads
2022-06-22 22:05:46,617 : INFO : worker thread finished; aw

2022-06-22 22:05:52,755 : INFO : worker thread finished; awaiting finish of 12 more threads
2022-06-22 22:05:52,800 : INFO : worker thread finished; awaiting finish of 11 more threads
2022-06-22 22:05:52,804 : INFO : worker thread finished; awaiting finish of 10 more threads
2022-06-22 22:05:52,807 : INFO : worker thread finished; awaiting finish of 9 more threads
2022-06-22 22:05:52,817 : INFO : worker thread finished; awaiting finish of 8 more threads
2022-06-22 22:05:52,827 : INFO : worker thread finished; awaiting finish of 7 more threads
2022-06-22 22:05:52,830 : INFO : worker thread finished; awaiting finish of 6 more threads
2022-06-22 22:05:52,835 : INFO : worker thread finished; awaiting finish of 5 more threads
2022-06-22 22:05:52,840 : INFO : worker thread finished; awaiting finish of 4 more threads
2022-06-22 22:05:52,845 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-06-22 22:05:52,851 : INFO : worker thread finished; awaiting finish of 2 more thre

2022-06-22 22:06:01,675 : INFO : Word2Vec lifecycle event {'msg': 'training on 2904060 raw words (1536904 effective words) took 14.8s, 104060 effective words/s', 'datetime': '2022-06-22T22:06:01.675761', 'gensim': '4.1.2', 'python': '3.9.12 (main, Apr  5 2022, 06:56:58) \n[GCC 7.5.0]', 'platform': 'Linux-5.13.0-48-generic-x86_64-with-glibc2.34', 'event': 'train'}
2022-06-22 22:06:01,676 : INFO : Word2Vec lifecycle event {'fname_or_handle': '/home/grace/walking/temporary/N.model', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2022-06-22T22:06:01.676404', 'gensim': '4.1.2', 'python': '3.9.12 (main, Apr  5 2022, 06:56:58) \n[GCC 7.5.0]', 'platform': 'Linux-5.13.0-48-generic-x86_64-with-glibc2.34', 'event': 'saving'}
2022-06-22 22:06:01,677 : INFO : not storing attribute cum_table
2022-06-22 22:06:01,802 : INFO : saved /home/grace/walking/temporary/N.model
2022-06-22 22:06:01,803 : INFO : loading Word2Vec object from /home/grace/walking/temporary/N.model


2022-06-22 22:06:10,506 : INFO : worker thread finished; awaiting finish of 12 more threads
2022-06-22 22:06:10,529 : INFO : worker thread finished; awaiting finish of 11 more threads
2022-06-22 22:06:10,530 : INFO : worker thread finished; awaiting finish of 10 more threads
2022-06-22 22:06:10,555 : INFO : worker thread finished; awaiting finish of 9 more threads
2022-06-22 22:06:10,562 : INFO : worker thread finished; awaiting finish of 8 more threads
2022-06-22 22:06:10,564 : INFO : worker thread finished; awaiting finish of 7 more threads
2022-06-22 22:06:10,573 : INFO : worker thread finished; awaiting finish of 6 more threads
2022-06-22 22:06:10,583 : INFO : worker thread finished; awaiting finish of 5 more threads
2022-06-22 22:06:10,604 : INFO : worker thread finished; awaiting finish of 4 more threads
2022-06-22 22:06:10,608 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-06-22 22:06:10,611 : INFO : worker thread finished; awaiting finish of 2 more thre

In [34]:
df_size_scores

size = 50  size = 100  size = 150  \
prop               option                                                     
state              crystalline            -0.514534   -0.521162   -0.528452   
                   semicrystalline        -0.525378   -0.522525   -0.532572   
                   amorphous              -0.530433   -0.537784   -0.541560   
                   liquid                 -0.554255   -0.572411   -0.578889   
composition        organic                -0.471406   -0.470268   -0.470822   
                   hybrid                 -0.504781   -0.504965   -0.519912   
                   inorganic              -0.571755   -0.571016   -0.573488   
                   metallic               -0.596098   -0.612154   -0.615341   
conjugation        conjugated             -0.499166   -0.488627   -0.479375   
                   non-conjugated         -0.585587   -0.581950   -0.572349   
porosity           microporous            -0.458115   -0.465096   -0.450530   
                   mesoporous             -0.499493   -0.489058   -0.500101   
                   non-porous             -0.510904   -0.504655   -0.506368   
form               particles              -0.570732   -0.571207   -0.571821   
                   nanoparticles          -0.573240   -0.577308   -0.576707   
                   solid                  -0.592764   -0.604625   -0.598723   
                   liquid                 -0.625151   -0.615209   -0.625027   
hydrophilicity     hydrophobic            -0.530010   -0.532685   -0.534180   
                   hydrophilic            -0.551290   -0.536797   -0.540018   
dispersibility     dispersible            -0.435396   -0.424993   -0.417863   
                   poorly__dispersible    -0.435501   -0.441395   -0.445287   
                   non-dispersible        -0.480238   -0.488372   -0.497911   
                   soluble                -0.545208   -0.567054   -0.559197   
conductivity       conducting             -0.438343   -0.423686   -0.422059   
                   semi-conducting        -0.485708   -0.486321   -0.487806   
                   insulating             -0.507723   -0.500117   -0.496404   
functional__role   light__absorber        -0.409145   -0.401315   -0.423246   
                   cocatalyst             -0.449460   -0.437441   -0.447769   
                   electron__scavenger    -0.450551   -0.462073   -0.464844   
                   hole__scavenger        -0.478547   -0.480177   -0.478883   
                   dye                    -0.488792   -0.485706   -0.496857   
                   solvent                -0.497551   -0.497127   -0.497083   
                   surfactant             -0.514191   -0.512180   -0.516213   
                   her__catalyst          -0.523171   -0.521836   -0.523350   
                   oer__catalyst          -0.534482   -0.535225   -0.531270   
                   overall__ws__catalyst  -0.575576   -0.582270   -0.603767   
                   ph__modifier                 NaN         NaN         NaN   
                   dielectric__modifier         NaN         NaN         NaN   
water__solubility  soluble                -0.381550   -0.374361   -0.376085   
                   insoluble              -0.475019   -0.497696   -0.478256   
                   sparingly__soluble     -0.551782   -0.565138   -0.566300   
functional__groups aromatic               -0.447674   -0.438218   -0.440793   
                   aliphatic              -0.448336   -0.453784   -0.442960   
                   carboxylic__acid       -0.459024   -0.472165   -0.482262   
                   ester                  -0.495151   -0.497493   -0.501743   
                   imine                  -0.522466   -0.522520   -0.526870   
                   sulfone                -0.524365   -0.534492   -0.527983   
                   triazine               -0.565179   -0.564797   -0.569479   

                                          size = 200       std  
prop               option 

In [21]:
df_size_infers.to_csv('/home/grace/walking/preturb_prop_infers/df_size_infers.csv')
df_size_scores.to_csv('/home/grace/walking/preturb_prop_scores/df_size_scores.csv')

In [22]:
df_window_infers = pd.DataFrame(index=pd.MultiIndex.from_tuples(mat_prop_tuples, names = ["mat","prop"]))
df_window_scores = pd.DataFrame(index=pd.MultiIndex.from_tuples(prop_tuples, names = ["prop","option"]))
window_models = {}
for i in window_arange:
    preturb(window = i)
    window_models[i] = embedding.dww2v(None)
    window_models[i].load_model(temp_path)
    
    infers = infer(window_models[i])
    df_window_infers["window = " + str(i)] = [prop for mat in infers for prop in mat] # turn lol into list
    
    df_window_scores["window = " + str(i)] = score(window_models[i])
df_window_scores["std"] = df_window_scores.std(axis = 1).tolist()

2022-06-21 14:23:46,445 : INFO : Training the model using the following parameters:
2022-06-21 14:23:46,446 : INFO : 	phrase_min_count: 10
2022-06-21 14:23:46,446 : INFO : 	size: 200
2022-06-21 14:23:46,446 : INFO : 	window: 2
2022-06-21 14:23:46,446 : INFO : 	min_count: 5
2022-06-21 14:23:46,447 : INFO : 	sg: True
2022-06-21 14:23:46,447 : INFO : 	hs: True
2022-06-21 14:23:46,447 : INFO : 	workers: 20
2022-06-21 14:23:46,448 : INFO : 	negative: 15
2022-06-21 14:23:46,448 : INFO : 	start_alpha: 0.001
2022-06-21 14:23:46,448 : INFO : 	end_alpha: 0.0001
2022-06-21 14:23:46,448 : INFO : 	subsample: 0.0001
2022-06-21 14:23:46,449 : INFO : 	batch: 5000
2022-06-21 14:23:46,449 : INFO : 	epochs: 5
2022-06-21 14:23:46,449 : INFO : The model will be saved in None
2022-06-21 14:23:46,449 : WARNING : Effective 'alpha' higher than previous training cycles
2022-06-21 14:23:46,450 : INFO : Word2Vec lifecycle event {'msg': 'training model with 20 workers on 10819 vocabulary and 200 features, using sg

2022-06-21 14:23:57,508 : INFO : EPOCH 4 - PROGRESS: at 59.29% examples, 89163 words/s, in_qsize 40, out_qsize 0
2022-06-21 14:23:58,231 : INFO : worker thread finished; awaiting finish of 19 more threads
2022-06-21 14:23:58,234 : INFO : worker thread finished; awaiting finish of 18 more threads
2022-06-21 14:23:58,286 : INFO : worker thread finished; awaiting finish of 17 more threads
2022-06-21 14:23:58,292 : INFO : worker thread finished; awaiting finish of 16 more threads
2022-06-21 14:23:58,301 : INFO : worker thread finished; awaiting finish of 15 more threads
2022-06-21 14:23:58,334 : INFO : worker thread finished; awaiting finish of 14 more threads
2022-06-21 14:23:58,336 : INFO : worker thread finished; awaiting finish of 13 more threads
2022-06-21 14:23:58,337 : INFO : worker thread finished; awaiting finish of 12 more threads
2022-06-21 14:23:58,357 : INFO : worker thread finished; awaiting finish of 11 more threads
2022-06-21 14:23:58,371 : INFO : worker thread finished; aw

2022-06-21 14:24:04,679 : INFO : worker thread finished; awaiting finish of 11 more threads
2022-06-21 14:24:04,682 : INFO : worker thread finished; awaiting finish of 10 more threads
2022-06-21 14:24:04,689 : INFO : worker thread finished; awaiting finish of 9 more threads
2022-06-21 14:24:04,697 : INFO : worker thread finished; awaiting finish of 8 more threads
2022-06-21 14:24:04,707 : INFO : worker thread finished; awaiting finish of 7 more threads
2022-06-21 14:24:04,716 : INFO : worker thread finished; awaiting finish of 6 more threads
2022-06-21 14:24:04,733 : INFO : worker thread finished; awaiting finish of 5 more threads
2022-06-21 14:24:04,736 : INFO : worker thread finished; awaiting finish of 4 more threads
2022-06-21 14:24:04,769 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-06-21 14:24:04,771 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-06-21 14:24:04,780 : INFO : worker thread finished; awaiting finish of 1 more threa

2022-06-21 14:24:14,107 : INFO : 4 Epoch(s) done. Loss: 1302550.0, LR: 0.000280748992241342
2022-06-21 14:24:15,178 : INFO : EPOCH 5 - PROGRESS: at 23.27% examples, 67027 words/s, in_qsize 40, out_qsize 0
2022-06-21 14:24:16,186 : INFO : EPOCH 5 - PROGRESS: at 60.15% examples, 89057 words/s, in_qsize 39, out_qsize 0
2022-06-21 14:24:16,933 : INFO : worker thread finished; awaiting finish of 19 more threads
2022-06-21 14:24:16,938 : INFO : worker thread finished; awaiting finish of 18 more threads
2022-06-21 14:24:16,960 : INFO : worker thread finished; awaiting finish of 17 more threads
2022-06-21 14:24:16,962 : INFO : worker thread finished; awaiting finish of 16 more threads
2022-06-21 14:24:16,971 : INFO : worker thread finished; awaiting finish of 15 more threads
2022-06-21 14:24:16,979 : INFO : worker thread finished; awaiting finish of 14 more threads
2022-06-21 14:24:16,991 : INFO : worker thread finished; awaiting finish of 13 more threads
2022-06-21 14:24:17,001 : INFO : worke

2022-06-21 14:24:23,377 : INFO : worker thread finished; awaiting finish of 13 more threads
2022-06-21 14:24:23,383 : INFO : worker thread finished; awaiting finish of 12 more threads
2022-06-21 14:24:23,388 : INFO : worker thread finished; awaiting finish of 11 more threads
2022-06-21 14:24:23,415 : INFO : worker thread finished; awaiting finish of 10 more threads
2022-06-21 14:24:23,422 : INFO : worker thread finished; awaiting finish of 9 more threads
2022-06-21 14:24:23,429 : INFO : worker thread finished; awaiting finish of 8 more threads
2022-06-21 14:24:23,438 : INFO : worker thread finished; awaiting finish of 7 more threads
2022-06-21 14:24:23,442 : INFO : worker thread finished; awaiting finish of 6 more threads
2022-06-21 14:24:23,456 : INFO : worker thread finished; awaiting finish of 5 more threads
2022-06-21 14:24:23,485 : INFO : worker thread finished; awaiting finish of 4 more threads
2022-06-21 14:24:23,486 : INFO : worker thread finished; awaiting finish of 3 more thr

...

2022-06-21 14:25:43,087 : INFO : worker thread finished; awaiting finish of 12 more threads
2022-06-21 14:25:43,098 : INFO : worker thread finished; awaiting finish of 11 more threads
2022-06-21 14:25:43,118 : INFO : worker thread finished; awaiting finish of 10 more threads
2022-06-21 14:25:43,125 : INFO : worker thread finished; awaiting finish of 9 more threads
2022-06-21 14:25:43,176 : INFO : worker thread finished; awaiting finish of 8 more threads
2022-06-21 14:25:43,190 : INFO : worker thread finished; awaiting finish of 7 more threads
2022-06-21 14:25:43,212 : INFO : worker thread finished; awaiting finish of 6 more threads
2022-06-21 14:25:43,216 : INFO : worker thread finished; awaiting finish of 5 more threads
2022-06-21 14:25:43,226 : INFO : worker thread finished; awaiting finish of 4 more threads
2022-06-21 14:25:43,227 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-06-21 14:25:43,229 : INFO : worker thread finished; awaiting finish of 2 more thre

2022-06-21 14:25:49,404 : INFO : The model will be saved in None
2022-06-21 14:25:49,405 : WARNING : Effective 'alpha' higher than previous training cycles
2022-06-21 14:25:49,405 : INFO : Word2Vec lifecycle event {'msg': 'training model with 20 workers on 10819 vocabulary and 200 features, using sg=1 hs=1 sample=0.0001 negative=15 window=8 shrink_windows=True', 'datetime': '2022-06-21T14:25:49.405591', 'gensim': '4.1.2', 'python': '3.9.12 (main, Apr  5 2022, 06:56:58) \n[GCC 7.5.0]', 'platform': 'Linux-5.13.0-48-generic-x86_64-with-glibc2.34', 'event': 'train'}
2022-06-21 14:25:50,481 : INFO : EPOCH 1 - PROGRESS: at 24.11% examples, 68754 words/s, in_qsize 39, out_qsize 1
2022-06-21 14:25:51,482 : INFO : EPOCH 1 - PROGRESS: at 60.15% examples, 88936 words/s, in_qsize 39, out_qsize 0
2022-06-21 14:25:52,150 : INFO : worker thread finished; awaiting finish of 19 more threads
2022-06-21 14:25:52,152 : INFO : worker thread finished; awaiting finish of 18 more threads
2022-06-21 14:25:52,1

2022-06-21 14:26:01,332 : INFO : worker thread finished; awaiting finish of 13 more threads
2022-06-21 14:26:01,348 : INFO : worker thread finished; awaiting finish of 12 more threads
2022-06-21 14:26:01,352 : INFO : worker thread finished; awaiting finish of 11 more threads
2022-06-21 14:26:01,362 : INFO : worker thread finished; awaiting finish of 10 more threads
2022-06-21 14:26:01,380 : INFO : worker thread finished; awaiting finish of 9 more threads
2022-06-21 14:26:01,381 : INFO : worker thread finished; awaiting finish of 8 more threads
2022-06-21 14:26:01,401 : INFO : worker thread finished; awaiting finish of 7 more threads
2022-06-21 14:26:01,402 : INFO : worker thread finished; awaiting finish of 6 more threads
2022-06-21 14:26:01,413 : INFO : worker thread finished; awaiting finish of 5 more threads
2022-06-21 14:26:01,419 : INFO : worker thread finished; awaiting finish of 4 more threads
2022-06-21 14:26:01,420 : INFO : worker thread finished; awaiting finish of 3 more thr

2022-06-21 14:26:07,574 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-06-21 14:26:07,575 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-06-21 14:26:07,578 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-06-21 14:26:07,584 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-06-21 14:26:07,585 : INFO : EPOCH - 1 : training on 580812 raw words (306868 effective words) took 2.9s, 104322 effective words/s
2022-06-21 14:26:07,585 : INFO : 1 Epoch(s) done. Loss: 1518200.375, LR: 0.000820748992241342
2022-06-21 14:26:08,600 : INFO : EPOCH 2 - PROGRESS: at 22.41% examples, 67802 words/s, in_qsize 40, out_qsize 0
2022-06-21 14:26:09,618 : INFO : EPOCH 2 - PROGRESS: at 58.44% examples, 88486 words/s, in_qsize 39, out_qsize 0
2022-06-21 14:26:10,231 : INFO : worker thread finished; awaiting finish of 19 more threads
2022-06-21 14:26:10,275 : INFO : worker thread finished; awaiting finish of 18 more threa

2022-06-21 14:26:19,389 : INFO : worker thread finished; awaiting finish of 13 more threads
2022-06-21 14:26:19,390 : INFO : worker thread finished; awaiting finish of 12 more threads
2022-06-21 14:26:19,397 : INFO : worker thread finished; awaiting finish of 11 more threads
2022-06-21 14:26:19,409 : INFO : worker thread finished; awaiting finish of 10 more threads
2022-06-21 14:26:19,421 : INFO : worker thread finished; awaiting finish of 9 more threads
2022-06-21 14:26:19,435 : INFO : worker thread finished; awaiting finish of 8 more threads
2022-06-21 14:26:19,463 : INFO : worker thread finished; awaiting finish of 7 more threads
2022-06-21 14:26:19,464 : INFO : worker thread finished; awaiting finish of 6 more threads
2022-06-21 14:26:19,474 : INFO : worker thread finished; awaiting finish of 5 more threads
2022-06-21 14:26:19,488 : INFO : worker thread finished; awaiting finish of 4 more threads
2022-06-21 14:26:19,489 : INFO : worker thread finished; awaiting finish of 3 more thr

In [23]:
df_window_infers.to_csv('/home/grace/walking/preturb_prop_infers/df_window_infers.csv')
df_window_scores.to_csv('/home/grace/walking/preturb_prop_stdevs/df_window_scores.csv')

In [24]:
df_workers_infers = pd.DataFrame(index=pd.MultiIndex.from_tuples(mat_prop_tuples, names = ["mat","prop"]))
df_workers_scores = pd.DataFrame(index=pd.MultiIndex.from_tuples(prop_tuples, names = ["prop","option"]))
workers_models = {}
for i in workers_arange:
    preturb(workers = i)
    workers_models[i] = embedding.dww2v(None)
    workers_models[i].load_model(temp_path)
    
    infers = infer(workers_models[i])
    df_workers_infers["workers = " + str(i)] = [prop for mat in infers for prop in mat] # turn lol into list
    
    df_workers_std["workers = " + str(i)] = score(workers_models[i])
df_workers_scores["std"] = df_workers_scores.std(axis = 1).tolist()

2022-06-21 14:26:19,752 : INFO : Training the model using the following parameters:
2022-06-21 14:26:19,753 : INFO : 	phrase_min_count: 10
2022-06-21 14:26:19,753 : INFO : 	size: 200
2022-06-21 14:26:19,754 : INFO : 	window: 8
2022-06-21 14:26:19,754 : INFO : 	min_count: 5
2022-06-21 14:26:19,754 : INFO : 	sg: True
2022-06-21 14:26:19,755 : INFO : 	hs: True
2022-06-21 14:26:19,755 : INFO : 	workers: 10
2022-06-21 14:26:19,756 : INFO : 	negative: 15
2022-06-21 14:26:19,756 : INFO : 	start_alpha: 0.001
2022-06-21 14:26:19,756 : INFO : 	end_alpha: 0.0001
2022-06-21 14:26:19,757 : INFO : 	subsample: 0.0001
2022-06-21 14:26:19,757 : INFO : 	batch: 5000
2022-06-21 14:26:19,758 : INFO : 	epochs: 5
2022-06-21 14:26:19,758 : INFO : The model will be saved in None
2022-06-21 14:26:19,759 : WARNING : Effective 'alpha' higher than previous training cycles
2022-06-21 14:26:19,759 : INFO : Word2Vec lifecycle event {'msg': 'training model with 20 workers on 10819 vocabulary and 200 features, using sg

2022-06-21 14:26:30,545 : INFO : EPOCH 4 - PROGRESS: at 61.92% examples, 94171 words/s, in_qsize 40, out_qsize 1
2022-06-21 14:26:31,169 : INFO : worker thread finished; awaiting finish of 19 more threads
2022-06-21 14:26:31,195 : INFO : worker thread finished; awaiting finish of 18 more threads
2022-06-21 14:26:31,200 : INFO : worker thread finished; awaiting finish of 17 more threads
2022-06-21 14:26:31,281 : INFO : worker thread finished; awaiting finish of 16 more threads
2022-06-21 14:26:31,298 : INFO : worker thread finished; awaiting finish of 15 more threads
2022-06-21 14:26:31,315 : INFO : worker thread finished; awaiting finish of 14 more threads
2022-06-21 14:26:31,334 : INFO : worker thread finished; awaiting finish of 13 more threads
2022-06-21 14:26:31,342 : INFO : worker thread finished; awaiting finish of 12 more threads
2022-06-21 14:26:31,349 : INFO : worker thread finished; awaiting finish of 11 more threads
2022-06-21 14:26:31,354 : INFO : worker thread finished; aw

2022-06-21 14:26:37,545 : INFO : worker thread finished; awaiting finish of 11 more threads
2022-06-21 14:26:37,547 : INFO : worker thread finished; awaiting finish of 10 more threads
2022-06-21 14:26:37,558 : INFO : worker thread finished; awaiting finish of 9 more threads
2022-06-21 14:26:37,560 : INFO : worker thread finished; awaiting finish of 8 more threads
2022-06-21 14:26:37,562 : INFO : worker thread finished; awaiting finish of 7 more threads
2022-06-21 14:26:37,568 : INFO : worker thread finished; awaiting finish of 6 more threads
2022-06-21 14:26:37,576 : INFO : worker thread finished; awaiting finish of 5 more threads
2022-06-21 14:26:37,591 : INFO : worker thread finished; awaiting finish of 4 more threads
2022-06-21 14:26:37,598 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-06-21 14:26:37,606 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-06-21 14:26:37,615 : INFO : worker thread finished; awaiting finish of 1 more threa

2022-06-21 14:26:48,410 : INFO : EPOCH 5 - PROGRESS: at 60.17% examples, 91260 words/s, in_qsize 39, out_qsize 0
2022-06-21 14:26:49,018 : INFO : worker thread finished; awaiting finish of 19 more threads
2022-06-21 14:26:49,042 : INFO : worker thread finished; awaiting finish of 18 more threads
2022-06-21 14:26:49,062 : INFO : worker thread finished; awaiting finish of 17 more threads
2022-06-21 14:26:49,121 : INFO : worker thread finished; awaiting finish of 16 more threads
2022-06-21 14:26:49,214 : INFO : worker thread finished; awaiting finish of 15 more threads
2022-06-21 14:26:49,226 : INFO : worker thread finished; awaiting finish of 14 more threads
2022-06-21 14:26:49,228 : INFO : worker thread finished; awaiting finish of 13 more threads
2022-06-21 14:26:49,229 : INFO : worker thread finished; awaiting finish of 12 more threads
2022-06-21 14:26:49,260 : INFO : worker thread finished; awaiting finish of 11 more threads
2022-06-21 14:26:49,263 : INFO : worker thread finished; aw

2022-06-21 14:26:55,480 : INFO : worker thread finished; awaiting finish of 11 more threads
2022-06-21 14:26:55,486 : INFO : worker thread finished; awaiting finish of 10 more threads
2022-06-21 14:26:55,489 : INFO : worker thread finished; awaiting finish of 9 more threads
2022-06-21 14:26:55,493 : INFO : worker thread finished; awaiting finish of 8 more threads
2022-06-21 14:26:55,505 : INFO : worker thread finished; awaiting finish of 7 more threads
2022-06-21 14:26:55,507 : INFO : worker thread finished; awaiting finish of 6 more threads
2022-06-21 14:26:55,517 : INFO : worker thread finished; awaiting finish of 5 more threads
2022-06-21 14:26:55,519 : INFO : worker thread finished; awaiting finish of 4 more threads
2022-06-21 14:26:55,524 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-06-21 14:26:55,527 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-06-21 14:26:55,528 : INFO : worker thread finished; awaiting finish of 1 more threa

...

2022-06-21 14:27:13,371 : INFO : worker thread finished; awaiting finish of 9 more threads
2022-06-21 14:27:13,372 : INFO : worker thread finished; awaiting finish of 8 more threads
2022-06-21 14:27:13,428 : INFO : worker thread finished; awaiting finish of 7 more threads
2022-06-21 14:27:13,455 : INFO : worker thread finished; awaiting finish of 6 more threads
2022-06-21 14:27:13,462 : INFO : worker thread finished; awaiting finish of 5 more threads
2022-06-21 14:27:13,478 : INFO : worker thread finished; awaiting finish of 4 more threads
2022-06-21 14:27:13,479 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-06-21 14:27:13,480 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-06-21 14:27:13,483 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-06-21 14:27:13,487 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-06-21 14:27:13,488 : INFO : EPOCH - 3 : training on 580812 raw words (307459 effectiv

2022-06-21 14:27:20,685 : INFO : EPOCH 1 - PROGRESS: at 27.52% examples, 79929 words/s, in_qsize 39, out_qsize 0
2022-06-21 14:27:21,722 : INFO : EPOCH 1 - PROGRESS: at 64.51% examples, 94508 words/s, in_qsize 39, out_qsize 0
2022-06-21 14:27:22,234 : INFO : worker thread finished; awaiting finish of 19 more threads
2022-06-21 14:27:22,264 : INFO : worker thread finished; awaiting finish of 18 more threads
2022-06-21 14:27:22,274 : INFO : worker thread finished; awaiting finish of 17 more threads
2022-06-21 14:27:22,357 : INFO : worker thread finished; awaiting finish of 16 more threads
2022-06-21 14:27:22,366 : INFO : worker thread finished; awaiting finish of 15 more threads
2022-06-21 14:27:22,372 : INFO : worker thread finished; awaiting finish of 14 more threads
2022-06-21 14:27:22,374 : INFO : worker thread finished; awaiting finish of 13 more threads
2022-06-21 14:27:22,404 : INFO : worker thread finished; awaiting finish of 12 more threads
2022-06-21 14:27:22,410 : INFO : worke

2022-06-21 14:27:31,221 : INFO : worker thread finished; awaiting finish of 6 more threads
2022-06-21 14:27:31,223 : INFO : worker thread finished; awaiting finish of 5 more threads
2022-06-21 14:27:31,237 : INFO : worker thread finished; awaiting finish of 4 more threads
2022-06-21 14:27:31,239 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-06-21 14:27:31,241 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-06-21 14:27:31,242 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-06-21 14:27:31,245 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-06-21 14:27:31,245 : INFO : EPOCH - 4 : training on 580812 raw words (307201 effective words) took 2.9s, 106183 effective words/s
2022-06-21 14:27:31,246 : INFO : 4 Epoch(s) done. Loss: 1418619.5, LR: 0.000280748992241342
2022-06-21 14:27:32,287 : INFO : EPOCH 5 - PROGRESS: at 20.76% examples, 61226 words/s, in_qsize 39, out_qsize 0
2022-06-21 14:27:33,325

2022-06-21 14:27:38,461 : INFO : EPOCH 2 - PROGRESS: at 22.37% examples, 66297 words/s, in_qsize 39, out_qsize 0
2022-06-21 14:27:39,527 : INFO : EPOCH 2 - PROGRESS: at 59.29% examples, 86737 words/s, in_qsize 39, out_qsize 0
2022-06-21 14:27:40,155 : INFO : worker thread finished; awaiting finish of 19 more threads
2022-06-21 14:27:40,172 : INFO : worker thread finished; awaiting finish of 18 more threads
2022-06-21 14:27:40,218 : INFO : worker thread finished; awaiting finish of 17 more threads
2022-06-21 14:27:40,223 : INFO : worker thread finished; awaiting finish of 16 more threads
2022-06-21 14:27:40,248 : INFO : worker thread finished; awaiting finish of 15 more threads
2022-06-21 14:27:40,259 : INFO : worker thread finished; awaiting finish of 14 more threads
2022-06-21 14:27:40,278 : INFO : worker thread finished; awaiting finish of 13 more threads
2022-06-21 14:27:40,283 : INFO : worker thread finished; awaiting finish of 12 more threads
2022-06-21 14:27:40,327 : INFO : worke

2022-06-21 14:27:49,464 : INFO : worker thread finished; awaiting finish of 6 more threads
2022-06-21 14:27:49,467 : INFO : worker thread finished; awaiting finish of 5 more threads
2022-06-21 14:27:49,475 : INFO : worker thread finished; awaiting finish of 4 more threads
2022-06-21 14:27:49,479 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-06-21 14:27:49,484 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-06-21 14:27:49,493 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-06-21 14:27:49,502 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-06-21 14:27:49,502 : INFO : EPOCH - 5 : training on 580812 raw words (307513 effective words) took 3.0s, 101546 effective words/s
2022-06-21 14:27:49,503 : INFO : 5 Epoch(s) done. Loss: 1425721.0, LR: 0.00010074899224134205
2022-06-21 14:27:49,504 : INFO : Word2Vec lifecycle event {'msg': 'training on 2904060 raw words (1537128 effective words) took 15.0s,

In [25]:
df_workers_infers.to_csv('/home/grace/walking/preturb_prop_infers/df_workers_infers.csv')
df_workers_scores.to_csv('/home/grace/walking/preturb_prop_scores/df_workers_scores.csv')

In [26]:
df_batch_infers = pd.DataFrame(index=pd.MultiIndex.from_tuples(mat_prop_tuples, names = ["mat","prop"]))
df_batch_scores = pd.DataFrame(index=pd.MultiIndex.from_tuples(prop_tuples, names = ["prop","option"]))
batch_models = {}
for i in batch_arange:
    preturb(batch = i)
    batch_models[i] = embedding.dww2v(None)
    batch_models[i].load_model(temp_path)
    
    batch = infer(batch_models[i])
    df_batch_infers["batch = " + str(i)] = [prop for mat in infers for prop in mat] # turn lol into list
    
    df_batch_scores["batch = " + str(i)] = score(batch_models[i])
df_batch_scores["std"] = df_batch_scores.std(axis = 1).tolist()

2022-06-21 14:27:49,769 : INFO : Training the model using the following parameters:
2022-06-21 14:27:49,769 : INFO : 	phrase_min_count: 10
2022-06-21 14:27:49,770 : INFO : 	size: 200
2022-06-21 14:27:49,770 : INFO : 	window: 8
2022-06-21 14:27:49,771 : INFO : 	min_count: 5
2022-06-21 14:27:49,771 : INFO : 	sg: True
2022-06-21 14:27:49,771 : INFO : 	hs: True
2022-06-21 14:27:49,772 : INFO : 	workers: 20
2022-06-21 14:27:49,772 : INFO : 	negative: 15
2022-06-21 14:27:49,772 : INFO : 	start_alpha: 0.001
2022-06-21 14:27:49,772 : INFO : 	end_alpha: 0.0001
2022-06-21 14:27:49,773 : INFO : 	subsample: 0.0001
2022-06-21 14:27:49,773 : INFO : 	batch: 1000
2022-06-21 14:27:49,773 : INFO : 	epochs: 5
2022-06-21 14:27:49,773 : INFO : The model will be saved in None
2022-06-21 14:27:49,774 : WARNING : Effective 'alpha' higher than previous training cycles
2022-06-21 14:27:49,774 : INFO : Word2Vec lifecycle event {'msg': 'training model with 20 workers on 10819 vocabulary and 200 features, using sg

2022-06-21 14:27:59,819 : INFO : EPOCH 4 - PROGRESS: at 25.81% examples, 77650 words/s, in_qsize 39, out_qsize 0
2022-06-21 14:28:00,835 : INFO : EPOCH 4 - PROGRESS: at 61.04% examples, 92238 words/s, in_qsize 39, out_qsize 0
2022-06-21 14:28:01,485 : INFO : worker thread finished; awaiting finish of 19 more threads
2022-06-21 14:28:01,531 : INFO : worker thread finished; awaiting finish of 18 more threads
2022-06-21 14:28:01,551 : INFO : worker thread finished; awaiting finish of 17 more threads
2022-06-21 14:28:01,569 : INFO : worker thread finished; awaiting finish of 16 more threads
2022-06-21 14:28:01,570 : INFO : worker thread finished; awaiting finish of 15 more threads
2022-06-21 14:28:01,574 : INFO : worker thread finished; awaiting finish of 14 more threads
2022-06-21 14:28:01,577 : INFO : worker thread finished; awaiting finish of 13 more threads
2022-06-21 14:28:01,637 : INFO : worker thread finished; awaiting finish of 12 more threads
2022-06-21 14:28:01,648 : INFO : worke

2022-06-21 14:28:08,040 : INFO : worker thread finished; awaiting finish of 13 more threads
2022-06-21 14:28:08,054 : INFO : worker thread finished; awaiting finish of 12 more threads
2022-06-21 14:28:08,061 : INFO : worker thread finished; awaiting finish of 11 more threads
2022-06-21 14:28:08,083 : INFO : worker thread finished; awaiting finish of 10 more threads
2022-06-21 14:28:08,084 : INFO : worker thread finished; awaiting finish of 9 more threads
2022-06-21 14:28:08,102 : INFO : worker thread finished; awaiting finish of 8 more threads
2022-06-21 14:28:08,103 : INFO : worker thread finished; awaiting finish of 7 more threads
2022-06-21 14:28:08,108 : INFO : worker thread finished; awaiting finish of 6 more threads
2022-06-21 14:28:08,121 : INFO : worker thread finished; awaiting finish of 5 more threads
2022-06-21 14:28:08,127 : INFO : worker thread finished; awaiting finish of 4 more threads
2022-06-21 14:28:08,135 : INFO : worker thread finished; awaiting finish of 3 more thr

2022-06-21 14:28:17,282 : INFO : EPOCH - 4 : training on 580812 raw words (307564 effective words) took 3.1s, 99138 effective words/s
2022-06-21 14:28:17,282 : INFO : 4 Epoch(s) done. Loss: 1419351.0, LR: 0.000280748992241342
2022-06-21 14:28:18,313 : INFO : EPOCH 5 - PROGRESS: at 22.41% examples, 67006 words/s, in_qsize 39, out_qsize 0
2022-06-21 14:28:19,326 : INFO : EPOCH 5 - PROGRESS: at 57.58% examples, 86838 words/s, in_qsize 39, out_qsize 0
2022-06-21 14:28:20,093 : INFO : worker thread finished; awaiting finish of 19 more threads
2022-06-21 14:28:20,094 : INFO : worker thread finished; awaiting finish of 18 more threads
2022-06-21 14:28:20,127 : INFO : worker thread finished; awaiting finish of 17 more threads
2022-06-21 14:28:20,154 : INFO : worker thread finished; awaiting finish of 16 more threads
2022-06-21 14:28:20,157 : INFO : worker thread finished; awaiting finish of 15 more threads
2022-06-21 14:28:20,172 : INFO : worker thread finished; awaiting finish of 14 more thre

2022-06-21 14:28:26,394 : INFO : worker thread finished; awaiting finish of 15 more threads
2022-06-21 14:28:26,403 : INFO : worker thread finished; awaiting finish of 14 more threads
2022-06-21 14:28:26,464 : INFO : worker thread finished; awaiting finish of 13 more threads
2022-06-21 14:28:26,468 : INFO : worker thread finished; awaiting finish of 12 more threads
2022-06-21 14:28:26,482 : INFO : worker thread finished; awaiting finish of 11 more threads
2022-06-21 14:28:26,492 : INFO : worker thread finished; awaiting finish of 10 more threads
2022-06-21 14:28:26,535 : INFO : worker thread finished; awaiting finish of 9 more threads
2022-06-21 14:28:26,540 : INFO : worker thread finished; awaiting finish of 8 more threads
2022-06-21 14:28:26,548 : INFO : worker thread finished; awaiting finish of 7 more threads
2022-06-21 14:28:26,554 : INFO : worker thread finished; awaiting finish of 6 more threads
2022-06-21 14:28:26,561 : INFO : worker thread finished; awaiting finish of 5 more t

...

2022-06-21 14:29:27,237 : INFO : worker thread finished; awaiting finish of 7 more threads
2022-06-21 14:29:27,238 : INFO : worker thread finished; awaiting finish of 6 more threads
2022-06-21 14:29:27,247 : INFO : worker thread finished; awaiting finish of 5 more threads
2022-06-21 14:29:27,254 : INFO : worker thread finished; awaiting finish of 4 more threads
2022-06-21 14:29:27,265 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-06-21 14:29:27,292 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-06-21 14:29:27,293 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-06-21 14:29:27,303 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-06-21 14:29:27,303 : INFO : EPOCH - 2 : training on 580812 raw words (307173 effective words) took 3.0s, 103993 effective words/s
2022-06-21 14:29:27,304 : INFO : 2 Epoch(s) done. Loss: 1528831.125, LR: 0.0006407489922413419
2022-06-21 14:29:28,332 : INFO : EPOCH 3 -

2022-06-21 14:29:36,154 : INFO : not storing attribute cum_table
2022-06-21 14:29:36,283 : INFO : saved /home/grace/walking/temporary/N.model
2022-06-21 14:29:36,284 : INFO : loading Word2Vec object from /home/grace/walking/temporary/N.model
2022-06-21 14:29:36,313 : INFO : loading wv recursively from /home/grace/walking/temporary/N.model.wv.* with mmap=None
2022-06-21 14:29:36,314 : INFO : setting ignored attribute cum_table to None
2022-06-21 14:29:36,407 : INFO : Word2Vec lifecycle event {'fname': '/home/grace/walking/temporary/N.model', 'datetime': '2022-06-21T14:29:36.407708', 'gensim': '4.1.2', 'python': '3.9.12 (main, Apr  5 2022, 06:56:58) \n[GCC 7.5.0]', 'platform': 'Linux-5.13.0-48-generic-x86_64-with-glibc2.34', 'event': 'loaded'}
2022-06-21 14:29:36,414 : INFO : Training the model using the following parameters:
2022-06-21 14:29:36,415 : INFO : 	phrase_min_count: 10
2022-06-21 14:29:36,415 : INFO : 	size: 200
2022-06-21 14:29:36,416 : INFO : 	window: 8
2022-06-21 14:29:36,4

2022-06-21 14:29:45,174 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-06-21 14:29:45,174 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-06-21 14:29:45,179 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-06-21 14:29:45,187 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-06-21 14:29:45,187 : INFO : EPOCH - 3 : training on 580812 raw words (307316 effective words) took 2.9s, 105140 effective words/s
2022-06-21 14:29:45,188 : INFO : 3 Epoch(s) done. Loss: 1473519.5, LR: 0.0004607489922413419
2022-06-21 14:29:46,199 : INFO : EPOCH 4 - PROGRESS: at 24.09% examples, 73505 words/s, in_qsize 39, out_qsize 0
2022-06-21 14:29:47,314 : INFO : EPOCH 4 - PROGRESS: at 60.15% examples, 87239 words/s, in_qsize 39, out_qsize 0
2022-06-21 14:29:47,927 : INFO : worker thread finished; awaiting finish of 19 more threads
2022-06-21 14:29:47,935 : INFO : worker thread finished; awaiting finish of 18 more thread

2022-06-21 14:29:54,194 : INFO : worker thread finished; awaiting finish of 18 more threads
2022-06-21 14:29:54,197 : INFO : worker thread finished; awaiting finish of 17 more threads
2022-06-21 14:29:54,200 : INFO : worker thread finished; awaiting finish of 16 more threads
2022-06-21 14:29:54,214 : INFO : worker thread finished; awaiting finish of 15 more threads
2022-06-21 14:29:54,218 : INFO : worker thread finished; awaiting finish of 14 more threads
2022-06-21 14:29:54,251 : INFO : worker thread finished; awaiting finish of 13 more threads
2022-06-21 14:29:54,277 : INFO : worker thread finished; awaiting finish of 12 more threads
2022-06-21 14:29:54,326 : INFO : worker thread finished; awaiting finish of 11 more threads
2022-06-21 14:29:54,362 : INFO : worker thread finished; awaiting finish of 10 more threads
2022-06-21 14:29:54,363 : INFO : worker thread finished; awaiting finish of 9 more threads
2022-06-21 14:29:54,389 : INFO : worker thread finished; awaiting finish of 8 mor

2022-06-21 14:30:03,644 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-06-21 14:30:03,660 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-06-21 14:30:03,667 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-06-21 14:30:03,672 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-06-21 14:30:03,672 : INFO : EPOCH - 4 : training on 580812 raw words (307559 effective words) took 3.0s, 101280 effective words/s
2022-06-21 14:30:03,673 : INFO : 4 Epoch(s) done. Loss: 1710913.0, LR: 0.000280748992241342
2022-06-21 14:30:04,729 : INFO : EPOCH 5 - PROGRESS: at 22.37% examples, 65264 words/s, in_qsize 40, out_qsize 0
2022-06-21 14:30:05,740 : INFO : EPOCH 5 - PROGRESS: at 58.43% examples, 87050 words/s, in_qsize 40, out_qsize 0
2022-06-21 14:30:06,451 : INFO : worker thread finished; awaiting finish of 19 more threads
2022-06-21 14:30:06,475 : INFO : worker thread finished; awaiting finish of 18 more threads

In [27]:
df_batch_infers.to_csv('/home/grace/walking/preturb_prop_infers/df_batch_infers.csv')
df_batch_scores.to_csv('/home/grace/walking/preturb_prop_scores/df_batch_scores.csv')

In [28]:
df_epochs_infers = pd.DataFrame(index=pd.MultiIndex.from_tuples(mat_prop_tuples, names = ["mat","prop"]))
df_epochs_scores = pd.DataFrame(index=pd.MultiIndex.from_tuples(prop_tuples, names = ["prop","option"]))
epochs_models = {}
for i in epochs_arange:
    preturb(epochs = i)
    epochs_models[i] = embedding.dww2v(None)
    epochs_models[i].load_model(temp_path)
    
    infers = infer(epochs_models[i])
    df_epochs_infers["epochs = " + str(i)] = [prop for mat in infers for prop in mat] # turn lol into list
    
    df_epochs_scores["epochs = " + str(i)] = score(epochs_models[i])
df_epochs_scores["std"] = df_epochs_scores.std(axis = 1).tolist()

2022-06-21 14:30:06,955 : INFO : Training the model using the following parameters:
2022-06-21 14:30:06,956 : INFO : 	phrase_min_count: 10
2022-06-21 14:30:06,956 : INFO : 	size: 200
2022-06-21 14:30:06,957 : INFO : 	window: 8
2022-06-21 14:30:06,957 : INFO : 	min_count: 5
2022-06-21 14:30:06,957 : INFO : 	sg: True
2022-06-21 14:30:06,958 : INFO : 	hs: True
2022-06-21 14:30:06,958 : INFO : 	workers: 20
2022-06-21 14:30:06,959 : INFO : 	negative: 15
2022-06-21 14:30:06,959 : INFO : 	start_alpha: 0.001
2022-06-21 14:30:06,960 : INFO : 	end_alpha: 0.0001
2022-06-21 14:30:06,960 : INFO : 	subsample: 0.0001
2022-06-21 14:30:06,960 : INFO : 	batch: 5000
2022-06-21 14:30:06,961 : INFO : 	epochs: 2
2022-06-21 14:30:06,961 : INFO : The model will be saved in None
2022-06-21 14:30:06,962 : WARNING : Effective 'alpha' higher than previous training cycles
2022-06-21 14:30:06,964 : INFO : Word2Vec lifecycle event {'msg': 'training model with 20 workers on 10819 vocabulary and 200 features, using sg

2022-06-21 14:30:13,234 : INFO : Word2Vec lifecycle event {'msg': 'training model with 20 workers on 10819 vocabulary and 200 features, using sg=1 hs=1 sample=0.0001 negative=15 window=8 shrink_windows=True', 'datetime': '2022-06-21T14:30:13.234147', 'gensim': '4.1.2', 'python': '3.9.12 (main, Apr  5 2022, 06:56:58) \n[GCC 7.5.0]', 'platform': 'Linux-5.13.0-48-generic-x86_64-with-glibc2.34', 'event': 'train'}
2022-06-21 14:30:14,286 : INFO : EPOCH 1 - PROGRESS: at 22.37% examples, 65423 words/s, in_qsize 40, out_qsize 0
2022-06-21 14:30:15,292 : INFO : EPOCH 1 - PROGRESS: at 56.69% examples, 84748 words/s, in_qsize 39, out_qsize 0
2022-06-21 14:30:15,910 : INFO : worker thread finished; awaiting finish of 19 more threads
2022-06-21 14:30:15,939 : INFO : worker thread finished; awaiting finish of 18 more threads
2022-06-21 14:30:15,984 : INFO : worker thread finished; awaiting finish of 17 more threads
2022-06-21 14:30:16,026 : INFO : worker thread finished; awaiting finish of 16 more t

2022-06-21 14:30:22,355 : INFO : loading wv recursively from /home/grace/walking/temporary/N.model.wv.* with mmap=None
2022-06-21 14:30:22,356 : INFO : setting ignored attribute cum_table to None
2022-06-21 14:30:22,447 : INFO : Word2Vec lifecycle event {'fname': '/home/grace/walking/temporary/N.model', 'datetime': '2022-06-21T14:30:22.447162', 'gensim': '4.1.2', 'python': '3.9.12 (main, Apr  5 2022, 06:56:58) \n[GCC 7.5.0]', 'platform': 'Linux-5.13.0-48-generic-x86_64-with-glibc2.34', 'event': 'loaded'}
2022-06-21 14:30:22,454 : INFO : Training the model using the following parameters:
2022-06-21 14:30:22,454 : INFO : 	phrase_min_count: 10
2022-06-21 14:30:22,455 : INFO : 	size: 200
2022-06-21 14:30:22,455 : INFO : 	window: 8
2022-06-21 14:30:22,456 : INFO : 	min_count: 5
2022-06-21 14:30:22,456 : INFO : 	sg: True
2022-06-21 14:30:22,456 : INFO : 	hs: True
2022-06-21 14:30:22,457 : INFO : 	workers: 20
2022-06-21 14:30:22,457 : INFO : 	negative: 15
2022-06-21 14:30:22,458 : INFO : 	sta

2022-06-21 14:30:31,233 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-06-21 14:30:31,235 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-06-21 14:30:31,235 : INFO : EPOCH - 3 : training on 580812 raw words (307209 effective words) took 2.9s, 105239 effective words/s
2022-06-21 14:30:31,236 : INFO : 3 Epoch(s) done. Loss: 1496870.25, LR: 0.0003259362403016774
2022-06-21 14:30:32,274 : INFO : EPOCH 4 - PROGRESS: at 22.34% examples, 66340 words/s, in_qsize 39, out_qsize 0
2022-06-21 14:30:33,314 : INFO : EPOCH 4 - PROGRESS: at 60.17% examples, 89092 words/s, in_qsize 39, out_qsize 2
2022-06-21 14:30:33,892 : INFO : worker thread finished; awaiting finish of 19 more threads
2022-06-21 14:30:33,945 : INFO : worker thread finished; awaiting finish of 18 more threads
2022-06-21 14:30:33,963 : INFO : worker thread finished; awaiting finish of 17 more threads
2022-06-21 14:30:33,973 : INFO : worker thread finished; awaiting finish of 16 more thr

2022-06-21 14:30:40,165 : INFO : worker thread finished; awaiting finish of 16 more threads
2022-06-21 14:30:40,173 : INFO : worker thread finished; awaiting finish of 15 more threads
2022-06-21 14:30:40,174 : INFO : worker thread finished; awaiting finish of 14 more threads
2022-06-21 14:30:40,186 : INFO : worker thread finished; awaiting finish of 13 more threads
2022-06-21 14:30:40,205 : INFO : worker thread finished; awaiting finish of 12 more threads
2022-06-21 14:30:40,235 : INFO : worker thread finished; awaiting finish of 11 more threads
2022-06-21 14:30:40,236 : INFO : worker thread finished; awaiting finish of 10 more threads
2022-06-21 14:30:40,249 : INFO : worker thread finished; awaiting finish of 9 more threads
2022-06-21 14:30:40,251 : INFO : worker thread finished; awaiting finish of 8 more threads
2022-06-21 14:30:40,256 : INFO : worker thread finished; awaiting finish of 7 more threads
2022-06-21 14:30:40,259 : INFO : worker thread finished; awaiting finish of 6 more 

...

2022-06-21 14:31:34,508 : INFO : worker thread finished; awaiting finish of 17 more threads
2022-06-21 14:31:34,521 : INFO : worker thread finished; awaiting finish of 16 more threads
2022-06-21 14:31:34,543 : INFO : worker thread finished; awaiting finish of 15 more threads
2022-06-21 14:31:34,554 : INFO : worker thread finished; awaiting finish of 14 more threads
2022-06-21 14:31:34,617 : INFO : worker thread finished; awaiting finish of 13 more threads
2022-06-21 14:31:34,635 : INFO : worker thread finished; awaiting finish of 12 more threads
2022-06-21 14:31:34,637 : INFO : worker thread finished; awaiting finish of 11 more threads
2022-06-21 14:31:34,661 : INFO : worker thread finished; awaiting finish of 10 more threads
2022-06-21 14:31:34,676 : INFO : worker thread finished; awaiting finish of 9 more threads
2022-06-21 14:31:34,682 : INFO : worker thread finished; awaiting finish of 8 more threads
2022-06-21 14:31:34,688 : INFO : worker thread finished; awaiting finish of 7 more

2022-06-21 14:31:43,481 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-06-21 14:31:43,483 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-06-21 14:31:43,492 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-06-21 14:31:43,492 : INFO : EPOCH - 5 : training on 580812 raw words (307849 effective words) took 3.0s, 103834 effective words/s
2022-06-21 14:31:43,493 : INFO : 5 Epoch(s) done. Loss: 1505606.5, LR: 0.0004379681201508388
2022-06-21 14:31:44,533 : INFO : EPOCH 6 - PROGRESS: at 23.27% examples, 68959 words/s, in_qsize 40, out_qsize 0
2022-06-21 14:31:45,556 : INFO : EPOCH 6 - PROGRESS: at 62.77% examples, 93842 words/s, in_qsize 40, out_qsize 0
2022-06-21 14:31:46,101 : INFO : worker thread finished; awaiting finish of 19 more threads
2022-06-21 14:31:46,134 : INFO : worker thread finished; awaiting finish of 18 more threads
2022-06-21 14:31:46,170 : INFO : worker thread finished; awaiting finish of 17 more threa

2022-06-21 14:31:52,320 : INFO : loading Word2Vec object from /home/grace/walking/temporary/N.model
2022-06-21 14:31:52,347 : INFO : loading wv recursively from /home/grace/walking/temporary/N.model.wv.* with mmap=None
2022-06-21 14:31:52,348 : INFO : setting ignored attribute cum_table to None
2022-06-21 14:31:52,439 : INFO : Word2Vec lifecycle event {'fname': '/home/grace/walking/temporary/N.model', 'datetime': '2022-06-21T14:31:52.439674', 'gensim': '4.1.2', 'python': '3.9.12 (main, Apr  5 2022, 06:56:58) \n[GCC 7.5.0]', 'platform': 'Linux-5.13.0-48-generic-x86_64-with-glibc2.34', 'event': 'loaded'}
2022-06-21 14:31:52,446 : INFO : Training the model using the following parameters:
2022-06-21 14:31:52,447 : INFO : 	phrase_min_count: 10
2022-06-21 14:31:52,447 : INFO : 	size: 200
2022-06-21 14:31:52,447 : INFO : 	window: 8
2022-06-21 14:31:52,447 : INFO : 	min_count: 5
2022-06-21 14:31:52,448 : INFO : 	sg: True
2022-06-21 14:31:52,448 : INFO : 	hs: True
2022-06-21 14:31:52,448 : INFO

2022-06-21 14:32:01,078 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-06-21 14:32:01,083 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-06-21 14:32:01,097 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-06-21 14:32:01,098 : INFO : EPOCH - 3 : training on 580812 raw words (307070 effective words) took 2.9s, 106215 effective words/s
2022-06-21 14:32:01,099 : INFO : 3 Epoch(s) done. Loss: 1519544.25, LR: 0.0007004161068007455
2022-06-21 14:32:02,139 : INFO : EPOCH 4 - PROGRESS: at 24.94% examples, 73736 words/s, in_qsize 39, out_qsize 0
2022-06-21 14:32:03,153 : INFO : EPOCH 4 - PROGRESS: at 62.77% examples, 93887 words/s, in_qsize 39, out_qsize 0
2022-06-21 14:32:03,683 : INFO : worker thread finished; awaiting finish of 19 more threads
2022-06-21 14:32:03,698 : INFO : worker thread finished; awaiting finish of 18 more threads
2022-06-21 14:32:03,740 : INFO : worker thread finished; awaiting finish of 17 more thre

2022-06-21 14:32:12,569 : INFO : worker thread finished; awaiting finish of 12 more threads
2022-06-21 14:32:12,610 : INFO : worker thread finished; awaiting finish of 11 more threads
2022-06-21 14:32:12,615 : INFO : worker thread finished; awaiting finish of 10 more threads
2022-06-21 14:32:12,616 : INFO : worker thread finished; awaiting finish of 9 more threads
2022-06-21 14:32:12,621 : INFO : worker thread finished; awaiting finish of 8 more threads
2022-06-21 14:32:12,676 : INFO : worker thread finished; awaiting finish of 7 more threads
2022-06-21 14:32:12,678 : INFO : worker thread finished; awaiting finish of 6 more threads
2022-06-21 14:32:12,687 : INFO : worker thread finished; awaiting finish of 5 more threads
2022-06-21 14:32:12,688 : INFO : worker thread finished; awaiting finish of 4 more threads
2022-06-21 14:32:12,692 : INFO : worker thread finished; awaiting finish of 3 more threads
2022-06-21 14:32:12,699 : INFO : worker thread finished; awaiting finish of 2 more thre

In [29]:
df_epochs_infers.to_csv('/home/grace/walking/preturb_prop_infers/df_epochs_infers.csv')
df_epochs_scores.to_csv('/home/grace/walking/preturb_prop_scores/df_epochs_scores.csv')